This is just 1 plate per experiment, 3 different dates.

ABCD etc are different.

Column 1 2 3 = TS121 
Column 4 5 6 = TS163 

Column 7 8 9 = TS121 
Column 10 11 12 = TS163 

For example:

A1 A2 A3 are the same

No drugs in the plates.

Plate content (Black Flat BioGreiner Plate)

Col 1-6 -Row A - H:
0, 1E2, 1E3, 1E4, 1E5, 1E6, 1E7, 1E8 nM Arabinose

Col 7-12 - Row A - H: 
0, 1.5E4, 3E4, 6E4, 1.25E5, 2.5E5, 5E5, 1E6 nM Arabinose
 

Files
All files represent the same biological experiment

NB_OD600_mEGFP = old format 
R98 & R97 = new format that will be used in the future

In [1]:
import os
def getSubFolders(folderPath):
    subFolders = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and '_stats' in fn:
                pth = os.path.join(root, fn)
                if root in subFolders:
                    continue
                subFolders.append(pth)
    return subFolders

In [21]:
folder_input = '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/od_gfp_data/dataset'
lf = getSubFolders(folder_input)
import os
g = dict()
for subFolder in lf:
    #print subFolder
    plate_name = os.path.split(subFolder)[1]
    g[plate_name]= subFolder

print len(g.keys())

3


In [22]:
def flatten_l(l0,l1):
    l0.extend(l1)
    return l0

def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]
    
def link_dict(dict1, dict2):    
    result = {key: flatten_l(as_list(dict1.get(key, 0)),as_list(dict2.get(key, 0)))
              for key in set(dict1) | set(dict2)}
    return result

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def mean_std_dict(dict0):
    import numpy as np
    result_m = {key+"_mean": np.mean(np.asarray(dict0.get(key,0))) for key in set(dict0)}
    result_std = {key+"_std": np.std(np.asarray(dict0.get(key,0))) for key in set(dict0)}
    return merge_two_dicts(result_m,result_std)

def getMeanStd(dic_growth):
    if len(dic_growth) > 1:
        dic_comb = dic_growth[0]
        for i in range(1,len(dic_growth)):                        
            dic_comb = link_dict(dic_growth[i], dic_comb)            
    else:
        print "only one value array, no mean and std"
        return dic_growth    
    result = mean_std_dict(dic_comb)
    return result



In [23]:
def getGrowthLagdGFP(stats_file):
    import csv
    m_growth = dict()
    m_lag = dict()
    m_mean_exp_dGFPdtOD = dict()
    m_mean_sta_dGFPdtOD = dict()
    m_mean_max_dGFPdtOD = dict()
    with open(stats_file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                #print "row_0",row
                line_count += 1
            else:
                #print row
                line_count += 1
                m_growth[row[0]] = float(row[1])
                m_lag[row[0]] = float(row[2])
                m_mean_exp_dGFPdtOD[row[0]] = float(row[3])
                m_mean_sta_dGFPdtOD[row[0]] = float(row[4])
                m_mean_max_dGFPdtOD[row[0]] = float(row[5])
    return m_growth, m_lag,m_mean_exp_dGFPdtOD,m_mean_sta_dGFPdtOD,m_mean_max_dGFPdtOD


In [24]:
g

{'20190408 - R97 (2) - SparkControl magellan Sheet 1_stats.csv': '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/od_gfp_data/dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/20190408 - R97 (2) - SparkControl magellan Sheet 1_stats.csv',
 'NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92_stats.csv': '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/od_gfp_data/dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92_stats.csv',
 'R98 version 2 - SparkControl magellan Sheet 1_stats.csv': '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/od_gfp_data/dataset/R98 version 2 - SparkControl magellan Sheet 1/R98 version 2 - SparkControl magellan Sheet 1_stats.csv'}

In [25]:
growth_rate = dict()
lag_phase = dict()

all_file = []
growth_all = []
lag_all = []
mean_exp_dGFPdtOD = []
mean_sta_dGFPdtOD = []
mean_max_dGFPdtOD = []

for k_group_ex in g:
    growth_rate[k_group_ex] = dict()
    lag_phase[k_group_ex] = dict()
    
    growth_rate[k_group_ex]['mixted_date'] = dict()
    lag_phase[k_group_ex]['mixted_date'] = dict()

    stats_file_p = g[k_group_ex]    
    growth_rate[stats_file_p]= dict()
    lag_phase[stats_file_p]= dict()        

    all_file.append(stats_file_p)
    dic_g, dic_la,m_mean_exp_dGFPdtOD,m_mean_sta_dGFPdtOD,m_mean_max_dGFPdtOD = getGrowthLagdGFP(stats_file_p)

    growth_all.append(dic_g)
    lag_all.append(dic_la)
    mean_exp_dGFPdtOD.append(m_mean_exp_dGFPdtOD)
    mean_sta_dGFPdtOD.append(m_mean_sta_dGFPdtOD)
    mean_max_dGFPdtOD.append(m_mean_max_dGFPdtOD)

        
        

In [26]:
def generateWellName():
    list1 = map(chr, range(65, 73))
    list2 = range(1,13,1)
    list2 = map(str, list2)  
    well_name = []
    for l in list1:
        for n in list2:
            well_name.append(l+n)
    return well_name

well_name = generateWellName()
print well_name

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12']


In [29]:
data=growth_all[0]

c = ['R98','R97','R92']
import pandas as pd
df = pd.DataFrame()

for idx, data in enumerate(growth_all):
    s = 'growh_rate_'+c[idx]    
    if df.empty:
        df = pd.DataFrame.from_dict(data,orient='index',columns=[s])
    else:
        df[s] = pd.Series(data)

for idx, data in enumerate(lag_all):
    s = 'lag_phase_'+c[idx]    
    df[s] = pd.Series(data)        
for idx, data in enumerate(mean_exp_dGFPdtOD):
    s = 'mean_exp_dGFPdtOD_'+c[idx]    
    df[s] = pd.Series(data)        
for idx, data in enumerate(mean_sta_dGFPdtOD):
    s = 'mean_sta_dGFPdtO_'+c[idx]    
    df[s] = pd.Series(data)        
for idx, data in enumerate(mean_max_dGFPdtOD):
    s = 'mean_max_dGFPdtOD_'+c[idx]    
    df[s] = pd.Series(data)        
        
        
df.index = well_name
df.to_csv(os.path.join(folder_input,'stats_growth_lag_gfp.csv'))
df

,growh_rate_R98,growh_rate_R97,growh_rate_R92,lag_phase_R98,lag_phase_R97,lag_phase_R92,mean_exp_dGFPdtOD_R98,mean_exp_dGFPdtOD_R97,mean_exp_dGFPdtOD_R92,mean_sta_dGFPdtO_R98,mean_sta_dGFPdtO_R97,mean_sta_dGFPdtO_R92,mean_max_dGFPdtOD_R98,mean_max_dGFPdtOD_R97,mean_max_dGFPdtOD_R92
A1,0.574665,0.574732,0.690465,3.324720,3.369194,1.016577,824.669016,891.727416,14358.029883,-16.706613,-318.010521,3446.274047,1265.082656,1367.920294,24687.785873
A2,0.542845,0.555115,0.625081,3.510402,2.942573,0.845290,1505.796751,3080.257979,32014.132217,-33.515674,572.024845,10965.771113,2513.790217,5668.109066,57140.293382
A3,0.539087,0.605404,0.647910,3.740790,3.080672,0.802567,1577.321668,2898.353096,32063.080701,-26.656185,211.372761,10366.589664,2559.148265,5051.866434,54688.487025
A4,0.532258,0.592506,0.684070,3.647112,3.023638,0.842496,1555.846011,2937.172366,28925.403715,-108.649489,-10184.850898,9582.458453,2796.745405,5152.132999,50096.538137
A5,0.637894,0.609864,0.813269,3.476673,4.188201,1.499569,817.507967,818.459259,16760.018229,-28.213326,-62.895258,3214.208475,1037.266120,971.193759,18184.624003
A6,0.684802,0.647423,0.889662,3.632352,3.995886,1.418695,820.710458,799.055831,15312.482680,-53.653763,-21.145860,1853.859589,994.475138,949.840227,17186.709807
A7,0.721044,0.659110,0.875548,4.097762,4.268501,2.331793,891.621437,760.167288,10004.286904,-10.116850,-353.784218,-464.488746,1003.873262,914.606701,12135.593496
A8,0.527765,0.575991,0.629566,3.433064,3.325877,0.784708,865.474842,1044.235888,13630.218559,-24.162207,-368.572588,1158.431279,1432.986549,1552.113100,25738.316109
A9,0.616638,0.572878,0.606941,3.215556,3.280144,0.762945,787.052336,950.479387,14370.956791,-23.351683,-334.608337,1303.282151,1256.663462,1473.989737,25412.045091
A10,0.875231,0.845860,0.810317,3.633380,2.989729,1.008811,35.176192,38.580682,398.967942,1.425019,6.462300,84.754883,101.103716,174.130075,946.388956
